In [1]:
import sys
from gene import Gene, GeneChain
from evolution import Evolution
from selector import TournamentSelector
from loader import load_results, load_trading_params, load_nontrading_params, load_indivs
from crosser import Cross10to100
from mutator import GausianMutator, MutRandomIndivs
from util import print_row, print_header, floor05
from fitness import evaluate, Evaluation
from Effects import KolyanEffects
from copy import deepcopy
import numpy as np
import pickle
import datetime

## Load data

In [2]:
spread = 0.0001

In [3]:
db_indivs = load_indivs()   # pandas object, for fitness function; db individuals have only non-trading parameters
# db_results = load_results() # list of Quotes object, for fitness function
trading_params = load_trading_params()  # pandas object, for precomputation
nontrading_params = load_nontrading_params()    # pandas object, for precomputation

db_indivs.head(5)

,0,1,2,3,4,5,6,7,8
0,3,21,5,0,1.34,0.09,0.38,0.38,0.16
1,4,21,5,0,1.42,0.10,0.40,0.40,0.33
2,5,21,5,0,1.42,0.21,0.40,0.40,0.33
3,5,21,5,0,1.42,0.69,0.40,0.40,0.33
4,5,21,5,0,1.43,0.70,0.40,0.40,0.33


In [4]:
trading_params.head(5)

,TYPE,MIN,MAX,DESCRIPTION
0,float64,-1.0,1.0,distance between order price and level
1,float64,0.5,2.0,stop loss
2,float64,0.5,2.5,stop out
3,float64,0.5,3.0,take profit
4,float64,-2.0,2.0,bu level


In [5]:
nontrading_params.head(6)

,TYPE,MIN_RANGE,DESCRIPTION
0,int64,2.0,bars on level after fracture
1,int64,4.0,bars before level behind
2,int64,2.0,bars between fracture and touch
3,int64,2.0,number of lps bigger than 0.5
4,float64,0.5,fracture rollback
5,float64,0.5,consolidation rollback


## Init base gene chain

In [6]:
base_gene_chain = GeneChain()
for index, row in nontrading_params.iterrows():
    dtype = np.dtype(row["TYPE"]).type
    is_interval = True if dtype != bool else False
    min_range = dtype(row["MIN_RANGE"])
    max_value = dtype(max(db_indivs[index]))
    min_value = dtype(min(db_indivs[index]))

    base_gene_chain.add(Gene(dtype=dtype, is_interval=is_interval,
                             min_value=min_value, max_value=max_value, min_range=min_range, ordering=-1))

for index, row in trading_params.iterrows():
    is_interval = False
    dtype = np.dtype(row["TYPE"]).type
    min_value = dtype(row["MIN"])
    max_value = dtype(row["MAX"])
    base_gene_chain.add(Gene(dtype=dtype, is_interval=is_interval, tags=["trading"],
                             min_value=min_value, max_value=max_value, ordering=-1))

for gene in base_gene_chain:
    print(gene)

0
1
2
3
4
5
6
7
8
gene params:
is interval: True
min value = 3
max value = 15
this gene has no value yet...
gene params:
is interval: True
min value = 2
max value = 43
this gene has no value yet...
gene params:
is interval: True
min value = 2
max value = 12
this gene has no value yet...
gene params:
is interval: True
min value = 0
max value = 3
this gene has no value yet...
gene params:
is interval: True
min value = 0.7
max value = 2.1
this gene has no value yet...
gene params:
is interval: True
min value = 0.0
max value = 0.8
this gene has no value yet...
gene params:
is interval: True
min value = 0.0
max value = 0.6
this gene has no value yet...
gene params:
is interval: True
min value = 0.0
max value = 0.6
this gene has no value yet...
gene params:
is interval: True
min value = 0.0
max value = 0.6
this gene has no value yet...
gene params:
is interval: False
min value = -1.0
max value = 1.0
this gene has no value yet...
gene params:
is interval: False
min value = 0.5
max value = 2.0

In [7]:
list((gene.type, gene._is_trading) for gene in base_gene_chain)

[(numpy.int64, False),
 (numpy.int64, False),
 (numpy.int64, False),
 (numpy.int64, False),
 (numpy.float64, False),
 (numpy.float64, False),
 (numpy.float64, False),
 (numpy.float64, False),
 (numpy.float64, False),
 (numpy.float64, True),
 (numpy.float64, True),
 (numpy.float64, True),
 (numpy.float64, True),
 (numpy.float64, True),
 (numpy.float64, True),
 (numpy.float64, True),
 (numpy.int64, True),
 (numpy.int64, True),
 (numpy.int64, True)]

## Init genetic algorithm

In [8]:
gen_len = len(base_gene_chain)
pop_size = 30
evolution = Evolution(
    MAX_GENS=1000,
    POP_SIZE=pop_size,
    evaluation=Evaluation(spread=0.0001),
    start_indiv_min_fitness=0.5,
    base_gene_chain=base_gene_chain,
    crosser=Cross10to100(indpb=(1/gen_len), tournsize=int(pop_size / 10), save_part=0.1),
    mutator=MutRandomIndivs(sigma=0.05, indiv_indpb=1, gene_indpb=(1/gen_len), save_part=0.1),
    effects=KolyanEffects(rep=0.1), 
    verbose=True)

gen_len = 19
Initial generation
fitness: 32.9 models: 317 max account value: 53.8 max drawdown: 23.8
fitness: 21.0 models: 383 max account value: 36.9 max drawdown: 16.3
fitness: 20.0 models: 106 max account value: 27.4 max drawdown: 11.2
fitness: 14.4 models: 135 max account value: 26.1 max drawdown: 18.0
fitness: 14.4 models: 135 max account value: 26.1 max drawdown: 18.0
fitness: 13.4 models: 177 max account value: 13.4 max drawdown: 7.3
fitness: 13.4 models: 177 max account value: 13.4 max drawdown: 7.3
fitness: 6.7 models: 181 max account value: 11.2 max drawdown: 15.6
fitness: 2.9 models: 173 max account value: 10.5 max drawdown: 13.5
fitness: -0.0 models: 115 max account value: 5.6 max drawdown: 15.2
fitness: -0.0 models: 115 max account value: 5.6 max drawdown: 15.2
fitness: -0.0 models: 115 max account value: 5.6 max drawdown: 15.2
fitness: -2.3 models: 163 max account value: 16.8 max drawdown: 19.1
fitness: -9.9 models: 119 max account value: 5.4 max drawdown: 15.3
fitness: -

## Run 

In [9]:
evolution.run()

evolution.print_generation_fitnesses()
for i, gene in enumerate(evolution.best()):
    print(i, gene)

Generation: 1 | Best fitness: 33.0 Models: 317 (32s.)
Fitnesses:
fitness: 32.9 models: 317 max account value: 53.8 max drawdown: 23.8
fitness: 23.3 models: 218 max account value: 26.2 max drawdown: 9.5
fitness: 21.0 models: 383 max account value: 36.9 max drawdown: 16.3
fitness: 7.6 models: 21 max account value: 10.4 max drawdown: 4.2
fitness: 7.0 models: 28 max account value: 8.5 max drawdown: 4.5
fitness: 6.4 models: 9 max account value: 6.4 max drawdown: 2.0
fitness: 5.5 models: 43 max account value: 9.4 max drawdown: 6.2
fitness: 4.4 models: 97 max account value: 11.6 max drawdown: 8.7
fitness: 2.6 models: 31 max account value: 3.6 max drawdown: 5.6
fitness: 1.5 models: 43 max account value: 9.7 max drawdown: 10.1
fitness: 0.0 models: 1 max account value: 0 max drawdown: 0
fitness: 0 models: 0 max account value: 0 max drawdown: 0
fitness: 0.0 models: 1 max account value: 0 max drawdown: 0
fitness: -0.6 models: 31 max account value: 5.0 max drawdown: 5.6
fitness: -1.2 models: 11 max

Generation: 6 | Best fitness: 35.0 Models: 318 (31s.)
Fitnesses:
fitness: 35.4 models: 318 max account value: 51.0 max drawdown: 20.3
fitness: 32.9 models: 317 max account value: 53.8 max drawdown: 23.8
fitness: 32.9 models: 317 max account value: 53.8 max drawdown: 23.8
fitness: 20.6 models: 20 max account value: 20.6 max drawdown: 1.6
fitness: 18.7 models: 16 max account value: 18.7 max drawdown: 1.7
fitness: 15.1 models: 37 max account value: 15.1 max drawdown: 3.4
fitness: 11.5 models: 9 max account value: 11.5 max drawdown: 1.8
fitness: 10.9 models: 26 max account value: 13.9 max drawdown: 3.4
fitness: 10.8 models: 11 max account value: 10.8 max drawdown: 1.8
fitness: 10.2 models: 8 max account value: 10.2 max drawdown: 0
fitness: 10.0 models: 5 max account value: 10.0 max drawdown: 0
fitness: 9.7 models: 14 max account value: 11.5 max drawdown: 1.8
fitness: 8.5 models: 22 max account value: 10.2 max drawdown: 3.4
fitness: 8.4 models: 45 max account value: 13.0 max drawdown: 7.0
f

Generation: 11 | Best fitness: 43.0 Models: 316 (31s.)
Fitnesses:
fitness: 43.1 models: 316 max account value: 54.0 max drawdown: 17.0
fitness: 36.1 models: 316 max account value: 53.8 max drawdown: 20.6
fitness: 36.1 models: 316 max account value: 53.8 max drawdown: 20.6
fitness: 12.4 models: 13 max account value: 12.4 max drawdown: 1.6
fitness: 10.5 models: 21 max account value: 13.2 max drawdown: 4.5
fitness: 10.2 models: 14 max account value: 11.2 max drawdown: 2.8
fitness: 8.8 models: 33 max account value: 13.4 max drawdown: 4.6
fitness: 8.6 models: 8 max account value: 8.6 max drawdown: 1.6
fitness: 6.7 models: 29 max account value: 8.7 max drawdown: 3.7
fitness: 6.5 models: 8 max account value: 8.0 max drawdown: 1.5
fitness: 6.4 models: 14 max account value: 8.0 max drawdown: 1.6
fitness: 5.4 models: 26 max account value: 8.4 max drawdown: 4.5
fitness: 4.5 models: 11 max account value: 9.0 max drawdown: 4.5
fitness: 4.5 models: 20 max account value: 6.0 max drawdown: 3.0
fitness

Generation: 16 | Best fitness: 53.0 Models: 329 (34s.)
Fitnesses:
fitness: 52.9 models: 329 max account value: 63.8 max drawdown: 18.4
fitness: 43.1 models: 316 max account value: 54.0 max drawdown: 17.0
fitness: 41.1 models: 330 max account value: 55.5 max drawdown: 20.0
fitness: 20.4 models: 41 max account value: 20.4 max drawdown: 3.4
fitness: 11.3 models: 28 max account value: 12.0 max drawdown: 4.5
fitness: 9.7 models: 11 max account value: 9.7 max drawdown: 1.8
fitness: 7.0 models: 37 max account value: 11.0 max drawdown: 4.5
fitness: 6.5 models: 20 max account value: 12.2 max drawdown: 5.7
fitness: 6.1 models: 9 max account value: 9.5 max drawdown: 3.4
fitness: 6.0 models: 35 max account value: 16.2 max drawdown: 10.2
fitness: 5.7 models: 16 max account value: 7.4 max drawdown: 1.7
fitness: 5.7 models: 16 max account value: 8.7 max drawdown: 3.0
fitness: 5.4 models: 13 max account value: 8.4 max drawdown: 3.0
fitness: 5.2 models: 12 max account value: 8.4 max drawdown: 3.2
fitne

Generation: 21 | Best fitness: 68.0 Models: 328 (44s.)
Fitnesses:
fitness: 68.3 models: 328 max account value: 78.3 max drawdown: 16.9
fitness: 68.3 models: 328 max account value: 78.3 max drawdown: 16.9
fitness: 68.3 models: 328 max account value: 78.3 max drawdown: 16.9
fitness: 11.1 models: 19 max account value: 11.1 max drawdown: 1.7
fitness: 11.1 models: 17 max account value: 11.1 max drawdown: 1.7
fitness: 9.6 models: 19 max account value: 11.2 max drawdown: 1.6
fitness: 6.6 models: 13 max account value: 8.2 max drawdown: 1.6
fitness: 3.4 models: 15 max account value: 6.3 max drawdown: 3.2
fitness: 3.3 models: 37 max account value: 10.6 max drawdown: 7.3
fitness: 3.0 models: 3 max account value: 3.0 max drawdown: 0
fitness: 2.8 models: 24 max account value: 4.5 max drawdown: 3.4
fitness: 2.8 models: 8 max account value: 2.8 max drawdown: 1.7
fitness: 2.8 models: 24 max account value: 6.8 max drawdown: 4.0
fitness: 2.0 models: 1 max account value: 2.0 max drawdown: 0
fitness: 2.0 

...

Generation: 186 | Best fitness: 170.0 Models: 361 (31s.)
Fitnesses:
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 39.6 models: 204 max account value: 47.8 max drawdown: 10.6
fitness: 29.8 models: 47 max account value: 29.8 max drawdown: 2.0
fitness: 21.2 models: 34 max account value: 23.1 max drawdown: 1.9
fitness: 18.2 models: 23 max account value: 18.2 max drawdown: 1.9
fitness: 18.0 models: 31 max account value: 18.0 max drawdown: 4.0
fitness: 17.5 models: 26 max account value: 17.5 max drawdown: 3.6
fitness: 17.2 models: 41 max account value: 19.1 max drawdown: 4.0
fitness: 15.5 models: 37 max account value: 15.5 max drawdown: 4.0
fitness: 13.9 models: 31 max account value: 15.7 max drawdown: 3.6
fitness: 12.4 models: 22 max account value: 12.4 max drawdown: 2.0
fitness: 11.4 models: 19 max account value: 11.4 m

Generation: 190 | Best fitness: 170.0 Models: 361 (30s.)
Fitnesses:
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 24.0 models: 46 max account value: 26.0 max drawdown: 2.4
fitness: 21.6 models: 40 max account value: 21.6 max drawdown: 1.8
fitness: 20.5 models: 35 max account value: 20.5 max drawdown: 5.1
fitness: 20.4 models: 35 max account value: 22.1 max drawdown: 1.7
fitness: 18.4 models: 66 max account value: 26.2 max drawdown: 7.8
fitness: 18.3 models: 27 max account value: 18.3 max drawdown: 3.8
fitness: 14.8 models: 29 max account value: 14.9 max drawdown: 2.1
fitness: 13.6 models: 28 max account value: 15.6 max drawdown: 2.0
fitness: 10.8 models: 39 max account value: 16.8 max drawdown: 6.0
fitness: 10.5 models: 26 max account value: 12.4 max drawdown: 1.9
fitness: 10.1 models: 29 max account value: 14.0 max

Generation: 194 | Best fitness: 170.0 Models: 361 (28s.)
Fitnesses:
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 32.3 models: 213 max account value: 41.8 max drawdown: 13.3
fitness: 19.4 models: 26 max account value: 19.4 max drawdown: 2.2
fitness: 17.7 models: 37 max account value: 17.7 max drawdown: 2.5
fitness: 14.9 models: 22 max account value: 14.9 max drawdown: 4.1
fitness: 14.7 models: 40 max account value: 14.7 max drawdown: 3.4
fitness: 13.7 models: 58 max account value: 19.4 max drawdown: 7.6
fitness: 10.8 models: 16 max account value: 10.8 max drawdown: 1.8
fitness: 10.8 models: 30 max account value: 16.9 max drawdown: 6.1
fitness: 9.6 models: 24 max account value: 15.2 max drawdown: 5.6
fitness: 9.2 models: 30 max account value: 13.6 max drawdown: 4.4
fitness: 8.5 models: 20 max account value: 9.0 max d

Generation: 198 | Best fitness: 170.0 Models: 361 (29s.)
Fitnesses:
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 170.5 models: 361 max account value: 174.2 max drawdown: 9.3
fitness: 14.0 models: 27 max account value: 14.0 max drawdown: 1.9
fitness: 13.4 models: 29 max account value: 15.4 max drawdown: 2.4
fitness: 13.3 models: 31 max account value: 15.2 max drawdown: 3.8
fitness: 12.0 models: 16 max account value: 12.0 max drawdown: 2.0
fitness: 11.4 models: 18 max account value: 11.4 max drawdown: 3.8
fitness: 10.8 models: 14 max account value: 10.8 max drawdown: 3.6
fitness: 10.4 models: 20 max account value: 10.5 max drawdown: 1.9
fitness: 8.2 models: 10 max account value: 8.2 max drawdown: 1.9
fitness: 7.9 models: 16 max account value: 7.9 max drawdown: 1.9
fitness: 7.7 models: 13 max account value: 7.7 max drawdown: 1.8
fitness: 7.2 models: 16 max account value: 9.0 max drawdow

KeyboardInterrupt: 